In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import cPickle as pickle

In [2]:
path = 'data/nyc_network_graph.pkl'
with open(path, 'r') as f:
    G = pickle.load(f)

In [3]:
zones = list(pd.read_csv("data/geohash.csv").geohash.values)
len(zones)

42

In [4]:
from simulator import FleetSimulator
from agent import Agent

simulator = FleetSimulator(G)
simulator.load_requests('temp/data/taxi_tripdata_2016-05.csv', 100000)
simulator.init_vehicles(10000)

agent = Agent(zones)

In [5]:
vehicles, requests, reward = simulator.forward()

In [7]:
from mapping_utils import plot_overmap
import geopandas as gpd

nyc = gpd.read_file("temp/data/taxi_zones/taxi_zones.shp").to_crs('+proj=latlon')

In [10]:
point_y = vehicles[vehicles.available==1].lat.values
point_x = vehicles[vehicles.available==1].lon.values
radius=0.001
# plot_overmap(point_x, point_y, radius, nyc)

In [6]:
%%time
assignments = agent.match(vehicles, requests)

CPU times: user 171 ms, sys: 153 ms, total: 324 ms
Wall time: 410 ms


/Users/odatakuma/anaconda/lib/python2.7/site-packages/pandas/core/index.py:1160: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future
  result = getitem(key)


In [7]:
%%time
N = 60
total_cost = 0
for i in range(N):
    vehicles, requests, reward = simulator.forward(assignments)
    total_cost += reward
    assignments = agent.match(vehicles, requests)
    print i, reward, len(assignments)
print reward

/Users/odatakuma/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


0 0.0 447
1 0.0 466
2 0.0 484
3 0.0 474
4 0.0 496
5 0.0 470
6 0.0 486
7 0.0 461
8 0.0 494
9 0.0 502
10 0.0 496
11 0.0 452
12 0.0 477
13 0.0 476
14 0.0 514
15 0.0 468
16 0.0 504
17 0.0 459
18 0.0 450
19 0.0 448
20 0.0 448
21 -251.317708082 470
22 -135.946137052 480
23 -405.598126835 467
24 -973.572983305 483
25 -813.576906884 514
26 -170.222552594 459
27 -244.606270901 482
28 -458.761010708 441
29 -724.924802086 458
30 -2045.56010041 478
31 -1266.7390322 462
32 -579.88184022 438
33 -1622.91192761 474
34 -392.466014071 449
35 -1120.8223514 496
36 -1309.40947466 450
37 -291.105906859 443
38 -84.0847806594 456
39 -555.09204331 495
40 -927.814745638 418
41 -1198.99542897 460
42 -1090.75821104 435
43 -558.589862642 459
44 -630.280458871 453
45 -1674.52561628 427
46 -690.672200944 454
47 -848.940178107 399
48 -1691.61099221 430
49 -510.506374175 462
50 -595.053404584 440
51 -712.210232609 381
52 -253.847402444 474
53 -822.395227229 415
54 -633.703994162 417
55 -808.221922322 436
56 -862.19330

In [14]:
point_y = vehicles[vehicles.available==1].lat.values
point_x = vehicles[vehicles.available==1].lon.values
# point_y = simulator.all_requests.pickup_latitude.values[:10000]
# point_x = simulator.all_requests.pickup_longitude.values[:10000]
radius=0.001
# plot_overmap(point_x, point_y, radius, nyc)